In [106]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
import re

In [107]:
airline = pd.read_csv('/Users/huyguy/Documents/Documents - Huy’s MacBook Pro/Projects/Forage/British Airline/Data/airline-reviews.csv')
seat = pd.read_csv('/Users/huyguy/Documents/Documents - Huy’s MacBook Pro/Projects/Forage/British Airline/Data/seat-reviews.csv')
lounge = pd.read_csv('/Users/huyguy/Documents/Documents - Huy’s MacBook Pro/Projects/Forage/British Airline/Data/lounge-reviews.csv')

In [108]:
# Expand Review Stats from dictionary to individual columns
def expand_review_stats(df):
    parsed_data = [ast.literal_eval(entry) for entry in df['Review Stats']]
    new_df = pd.json_normalize(parsed_data)
    return pd.concat([df, new_df], axis=1)

In [109]:
airline = expand_review_stats(airline)
seat = expand_review_stats(seat)
lounge = expand_review_stats(lounge)

In [110]:
# Remove tick mark icon in review contents
def remove_trip_verification(review):
    '''Remove unwanted phrases'''
    if '✅ Trip Verified |' in review:
        review = review.replace('✅ Trip Verified |', '')
    if 'Not Verified |' in review:
        review = review.replace('Not Verified |', '')
    return review

In [111]:
# Remove unwanted phrases from reviews
airline['Review Body'] = airline['Review Body'].apply(remove_trip_verification)
seat['Review Body'] = seat['Review Body'].apply(remove_trip_verification)
lounge['Review Body'] = lounge['Review Body'].apply(remove_trip_verification)

In [112]:
# Clean review contents
def clean(text):
    '''Remove non alphabetical characters'''
    # Replace non-alphabetic characters with a single space
    text = re.sub('[^A-Za-z]+', ' ', text)   
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Remove leading/trailing spaces
    text = text.strip()
    return text

In [113]:
airline['Review Body'] = airline['Review Body'].apply(clean)
seat['Review Body'] = seat['Review Body'].apply(clean)
lounge['Review Body'] = lounge['Review Body'].apply(clean)

In [114]:
airline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3903 entries, 0 to 3902
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Title                   3903 non-null   object 
 1   Author                  3903 non-null   object 
 2   Date Published          3903 non-null   object 
 3   Review Body             3903 non-null   object 
 4   Review Stats            3903 non-null   object 
 5   Type Of Traveller       3132 non-null   object 
 6   Seat Type               3901 non-null   object 
 7   Route                   3127 non-null   object 
 8   Date Flown              3125 non-null   object 
 9   Seat Comfort            3777 non-null   float64
 10  Cabin Staff Service     3762 non-null   float64
 11  Food & Beverages        3468 non-null   float64
 12  Inflight Entertainment  2668 non-null   float64
 13  Ground Service          3053 non-null   float64
 14  Value For Money         3903 non-null   

In [116]:
# Adjust data type and make data for accessible
airline['Date Published'] = pd.to_datetime(airline['Date Published'], errors='coerce')
airline['Recommended'] = airline['Recommended'].map({'Yes': 1, 'No': 0})
airline["From"] = [x.split('to')[0].strip() if isinstance(x, str) else None for x in airline["Route"]]
airline["To"] = [
    x.split('to')[1].strip() if isinstance(x, str) and len(x.split('to')) > 1 else None 
    for x in airline["Route"]
]

In [117]:
# Drop columns
airline = airline.drop(columns = ['Route', 'Review Stats'], axis = 1)

In [120]:
lounge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              421 non-null    object 
 1   Author             421 non-null    object 
 2   Date Published     421 non-null    object 
 3   Review Body        421 non-null    object 
 4   Review Stats       421 non-null    object 
 5   Lounge Name        270 non-null    object 
 6   Airport            416 non-null    object 
 7   Type Of Lounge     393 non-null    object 
 8   Date Visit         188 non-null    object 
 9   Type Of Traveller  290 non-null    object 
 10  Comfort            419 non-null    float64
 11  Cleanliness        419 non-null    float64
 12  Bar & Beverages    409 non-null    float64
 13  Catering           412 non-null    float64
 14  Washrooms          367 non-null    float64
 15  Staff Service      405 non-null    float64
 16  Recommended        421 non

In [121]:
lounge.head()

,Title,Author,Date Published,Review Body,Review Stats,Lounge Name,Airport,Type Of Lounge,Date Visit,Type Of Traveller,Comfort,Cleanliness,Bar & Beverages,Catering,Washrooms,Staff Service,Recommended,Wifi Connectivity
0,"""there is not enough space""",E Carmere,2024-03-14,The lounge is clean but the seats are worn and...,"{'Lounge Name': 'Business Class', 'Airport': '...",Business Class,Barbados Grantley Adams Airport,Business Class,March 2024,Business,3.0,3.0,3.0,3.0,1.0,3.0,yes,NaN
1,"""Staff were welcoming and friendly""",Tony Maddern,2023-12-21,The lounge is very spacious with a wide variet...,"{'Airport': 'London Heathrow Airport', 'Type O...",NaN,London Heathrow Airport,Business Class,December 2023,Business,5.0,5.0,4.0,5.0,5.0,5.0,yes,5.0
2,"""rotten and totally incompetent airline""",S Neale,2023-09-28,Tatty and uncared for the BA business lounge i...,"{'Lounge Name': 'South Terminal', 'Airport': '...",South Terminal,London Gatwick Airport,Business Class,September 2023,Business,1.0,1.0,2.0,2.0,1.0,1.0,no,3.0
3,"""BA staff who are rude, abrasive""",Peter Glasier,2023-09-01,Crowded or rather overcrowded as usual The cat...,"{'Lounge Name': 'First Lounge - T5', 'Airport'...",First Lounge - T5,London Heathrow Airport,First Class,NaN,NaN,2.0,2.0,3.0,2.0,1.0,2.0,no,2.0
4,"""full service has been resumed""",Richard Hodges,2023-06-20,Flew London to Kalamata and as a Silver Saphir...,"{'Lounge Name': 'T5 Galleries South', 'Airport...",T5 Galleries South,London Heathrow Airport,Business Class,June 2023,Business,4.0,4.0,5.0,5.0,4.0,NaN,yes,NaN


In [122]:
# Similar work with airline
lounge['Date Published'] = pd.to_datetime(lounge['Date Published'], errors='coerce')
lounge = lounge.drop(columns = ['Review Stats'], axis = 1)

In [123]:
seat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              200 non-null    object 
 1   Author             200 non-null    object 
 2   Date Published     200 non-null    object 
 3   Review Body        200 non-null    object 
 4   Review Stats       200 non-null    object 
 5   Seat Type          200 non-null    object 
 6   Aircraft Type      200 non-null    object 
 7   Seat Layout        200 non-null    object 
 8   Date Flown         123 non-null    object 
 9   Type Of Traveller  123 non-null    object 
 10  Seat Legroom       190 non-null    float64
 11  Seat Recline       190 non-null    float64
 12  Seat Width         190 non-null    float64
 13  Aisle Space        190 non-null    float64
 14  Viewing Tv Screen  164 non-null    float64
 15  Power Supply       61 non-null     float64
 16  Seat Storage       122 non

In [126]:
seat.head(5)

,Title,Author,Date Published,Review Body,Review Stats,Seat Type,Aircraft Type,Seat Layout,Date Flown,Type Of Traveller,...,Aisle Space,Viewing Tv Screen,Power Supply,Seat Storage,Recommended,Sleep Comfort,Sitting Comfort,Seat/bed Width,Seat/bed Length,Seat Privacy
0,""" seat that doesn't recline by default""",E Junior,2024-12-19,Was sat on that famous row seat H Summary hour...,"{'Seat Type': 'Economy Class', 'Aircraft Type'...",Economy Class,A350-1000,3x3x3,December 2024,Solo Leisure,...,1.0,3.0,3.0,1.0,no,NaN,NaN,NaN,NaN,NaN
1,"""too many seats in each row""",M Stanfield,2024-10-24,Recently flew BA s nonstop between Boston and ...,"{'Seat Type': 'Business Class', 'Aircraft Type...",Business Class,A380,2x4x2,October 2024,Business,...,NaN,NaN,NaN,5.0,no,1.0,4.0,3.0,5.0,1.0
2,"""Seats were both very worn""",Steven Tempest,2024-09-02,Sat in seats a outbound to Budapest and A retu...,"{'Seat Type': 'Economy Class', 'Aircraft Type'...",Economy Class,A320,3X3,September 2024,Couple Leisure,...,3.0,NaN,1.0,1.0,no,NaN,NaN,NaN,NaN,NaN
3,"""One of the worst business class seats""",Ruben Gindic,2024-05-16,One of the worst business class seats tiny sea...,"{'Seat Type': 'Business Class', 'Aircraft Type...",Business Class,Boeing 787,2x3x2,May 2024,Leisure,...,NaN,NaN,1.0,5.0,no,1.0,2.0,1.0,3.0,4.0
4,"""left with a horrible back pain""",A Ahmed,2024-05-07,BA doesn t seem to understand why its customer...,"{'Seat Type': 'Economy Class', 'Aircraft Type'...",Economy Class,A320,3x3,April 2024,Couple Leisure,...,3.0,NaN,1.0,1.0,no,NaN,NaN,NaN,NaN,NaN


In [127]:
airline.to_csv('airline_processed.csv', index=False)
lounge.to_csv('lounge_processed.csv', index=False)
seat.to_csv('seat_processed.csv', index=False)